In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',50)

from dotenv import load_dotenv
import os
load_dotenv() 
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")
query ="select * from ppm;"
df = pd.read_sql(query, engine)

In [48]:
from datetime import datetime, timedelta

def replace_dates_with_last_150_days(df, date_column='Date'):
    # Get the 150 most recent dates from today going backward
    today = datetime.today().date()
    new_dates = pd.date_range(end=today, periods=150).date[::-1]  # Reverse to go backwards

    # Get the 150 unique old dates (sorted to keep order consistent)
    unique_old_dates = sorted(df['Date'].dropna().unique())[:150]

    # Create a mapping from old dates to new dates
    date_map = dict(zip(unique_old_dates, new_dates))

    # Replace the old dates with the new ones
    df['Date'] = df['Date'].map(date_map).fillna(df['Date'])  # keep non-mapped as is

    return df



In [ ]:
#use function to update old date to new dates
df = replace_dates_with_last_150_days(df, 'Date') 


In [ ]:
#check updated dates
df['Date'].agg(['min','max'])

min    2025-01-13
max    2025-06-11
Name: Date, dtype: object

In [52]:
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = 'last_150days_production'
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")


In [53]:
df.to_sql("production", con=engine, if_exists='replace', index=False)

3180